<a href="https://colab.research.google.com/github/NickolasFerraz1/GS-GOV/blob/main/GS_Governan%C3%A7a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Geração de Dados e Análise de Dados

In [20]:
!pip install -U kaleido

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import os

def gerar_dados_simulados(num_registros=1000):
    """
    Gera um DataFrame com dados de logs de emergência e o salva como um arquivo CSV.
    """
    print("--- Iniciando a Geração de Dados Simulados ---")

    data_inicio = datetime(2025, 5, 1)
    categorias = ['Parada Cardíaca', 'Acidente de Trânsito', 'Queda', 'Incêndio', 'Intoxicação']
    bairros = ['Centro', 'Jardim das Flores', 'Vila Industrial', 'Parque das Nações', 'Praia dos Amores']
    dados = []

    for i in range(num_registros):
        timestamp_inicio = data_inicio + timedelta(minutes=np.random.randint(0, 30*24*60))
        categoria_real = np.random.choice(categorias)
        bairro = np.random.choice(bairros)

        # Simula a acurácia da IA (90% de chance de acertar)
        if np.random.rand() < 0.9:
            categoria_ia = categoria_real
        else:
            categoria_ia = np.random.choice([c for c in categorias if c != categoria_real])

        # Simula os tempos de processamento em segundos
        tempo_classificacao_ia = timestamp_inicio + timedelta(seconds=np.random.uniform(5, 20))

        # Simula um GARGALO: Acidentes de trânsito e chamadas da 'Vila Industrial' demoram mais
        delay_extra_segundos = 0
        if categoria_real == 'Acidente de Trânsito':
            delay_extra_segundos += np.random.uniform(60, 120)
        if bairro == 'Vila Industrial':
            delay_extra_segundos += np.random.uniform(30, 90)

        tempo_despacho = tempo_classificacao_ia + timedelta(seconds=np.random.uniform(30, 90) + delay_extra_segundos)

        dados.append({
            'id_chamada': 1000 + i,
            'timestamp_inicio': timestamp_inicio,
            'bairro': bairro,
            'categoria_real': categoria_real,
            'categoria_ia': categoria_ia,
            'categoria_humano': categoria_real,
            'timestamp_classificacao_ia': tempo_classificacao_ia,
            'timestamp_despacho': tempo_despacho,
        })

    df_simulado = pd.DataFrame(dados)
    nome_arquivo = 'logs_emergencia_simulados.csv'
    df_simulado.to_csv(nome_arquivo, index=False)

    print(f"✅ Arquivo '{nome_arquivo}' gerado com sucesso com {len(df_simulado)} registros.")
    print("Amostra dos dados gerados:")
    print(df_simulado.head())

    return nome_arquivo

def analisar_logs(nome_arquivo):
    """
    Carrega o arquivo de logs, calcula indicadores e gera gráficos para análise.
    """
    print("\n--- Iniciando a Análise dos Dados ---")

    df = pd.read_csv(nome_arquivo)

    # Converte as colunas de texto para o formato de data e hora
    for col in ['timestamp_inicio', 'timestamp_classificacao_ia', 'timestamp_despacho']:
        df[col] = pd.to_datetime(df[col])

    # --- Cálculo dos Indicadores ---
    df['tempo_resposta_ia'] = (df['timestamp_classificacao_ia'] - df['timestamp_inicio']).dt.total_seconds()
    df['tempo_total_despacho'] = (df['timestamp_despacho'] - df['timestamp_inicio']).dt.total_seconds()
    df['tempo_total_resposta'] = (df['timestamp_classificacao_ia'] - df['timestamp_inicio']).dt.total_seconds()

    tempo_medio_resposta_ia = df['tempo_resposta_ia'].mean()
    acuracia_ia = (df['categoria_ia'] == df['categoria_humano']).mean() * 100
    tempo_medio_despacho = df['tempo_total_despacho'].mean()

    print("\n--- Indicadores de Performance ---")
    print(f"📈 Tempo Médio de Resposta da IA: {tempo_medio_resposta_ia:.2f} segundos")
    print(f"🎯 Acurácia na Classificação de Emergências: {acuracia_ia:.2f}%")
    print(f"🚑 Tempo Médio Total para Despacho: {tempo_medio_despacho:.2f} segundos")

    # --- Identificação de Gargalos com Visualizações ---
    print("\n--- Gerando Gráficos para Análise de Gargalos ---")

    # Gráfico 1: Histograma
    fig_hist = px.histogram(df, x='tempo_total_despacho', title='Distribuição do Tempo de Despacho (Geral)', labels={'tempo_total_despacho': 'Tempo (segundos)'})
    fig_hist.show()

    # Gráfico 2: Histograma do Tempo de Resposta da IA
    # Este novo gráfico foca apenas no tempo que a IA leva para classificar
    fig_resposta_ia = px.histogram(
        df,
        x='tempo_resposta_ia',
        nbins=40,
        title='Distribuição do Tempo de Resposta da IA',
        labels={'tempo_resposta_ia': 'Tempo de Resposta da IA (segundos)'}
    )
    fig_resposta_ia.show()

    # Gráfico 3: Boxplot por Categoria
    fig_box_cat = px.box(df, x='categoria_humano', y='tempo_total_despacho', title='Análise de Gargalos por Tipo de Emergência', labels={'categoria_humano': 'Tipo de Emergência', 'tempo_total_despacho': 'Tempo (segundos)'})
    fig_box_cat.show()

    # Gráfico 4: Boxplot por Bairro
    fig_box_bairro = px.box(df, x='bairro', y='tempo_total_despacho', title='Análise de Gargalos por Bairro', labels={'bairro': 'Bairro', 'tempo_total_despacho': 'Tempo (segundos)'})
    fig_box_bairro.show()


# --- Execução Principal ---
if __name__ == '__main__':
    # Passo 1: Gera o arquivo de dados
    arquivo_de_logs = gerar_dados_simulados()

    # Passo 2: Analisa o arquivo gerado
    analisar_logs(arquivo_de_logs)

--- Iniciando a Geração de Dados Simulados ---
✅ Arquivo 'logs_emergencia_simulados.csv' gerado com sucesso com 1000 registros.
Amostra dos dados gerados:
   id_chamada    timestamp_inicio             bairro   categoria_real  \
0        1000 2025-05-17 15:04:00   Praia dos Amores  Parada Cardíaca   
1        1001 2025-05-16 03:22:00   Praia dos Amores         Incêndio   
2        1002 2025-05-17 21:56:00    Vila Industrial         Incêndio   
3        1003 2025-05-23 17:18:00             Centro  Parada Cardíaca   
4        1004 2025-05-09 16:59:00  Jardim das Flores         Incêndio   

      categoria_ia categoria_humano timestamp_classificacao_ia  \
0  Parada Cardíaca  Parada Cardíaca 2025-05-17 15:04:16.237857   
1         Incêndio         Incêndio 2025-05-16 03:22:09.961943   
2         Incêndio         Incêndio 2025-05-17 21:56:07.141757   
3  Parada Cardíaca  Parada Cardíaca 2025-05-23 17:18:05.083173   
4      Intoxicação         Incêndio 2025-05-09 16:59:05.537261   

         

## Geração de Relatório

In [3]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.6 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import os
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

# ---------------------- GERAR DADOS SIMULADOS ----------------------
def gerar_dados_simulados(num_registros=1000):
    """
    Gera um DataFrame com dados de logs de emergência e o salva como um arquivo CSV.
    """
    print("--- Iniciando a Geração de Dados Simulados ---")

    data_inicio = datetime(2025, 5, 1)
    categorias = ['Parada Cardíaca', 'Acidente de Trânsito', 'Queda', 'Incêndio', 'Intoxicação']
    bairros = ['Centro', 'Jardim das Flores', 'Vila Industrial', 'Parque das Nações', 'Praia dos Amores']
    dados = []

    for i in range(num_registros):
        timestamp_inicio = data_inicio + timedelta(minutes=np.random.randint(0, 30*24*60))
        categoria_real = np.random.choice(categorias)
        bairro = np.random.choice(bairros)

        # Simula a acurácia da IA (90% de chance de acertar)
        if np.random.rand() < 0.9:
            categoria_ia = categoria_real
        else:
            categoria_ia = np.random.choice([c for c in categorias if c != categoria_real])

        # Simula os tempos de processamento em segundos
        tempo_classificacao_ia = timestamp_inicio + timedelta(seconds=np.random.uniform(5, 20))

        # Simula um GARGALO: Acidentes de trânsito e chamadas da 'Vila Industrial' demoram mais
        delay_extra_segundos = 0
        if categoria_real == 'Acidente de Trânsito':
            delay_extra_segundos += np.random.uniform(60, 120)
        if bairro == 'Vila Industrial':
            delay_extra_segundos += np.random.uniform(30, 90)

        tempo_despacho = tempo_classificacao_ia + timedelta(seconds=np.random.uniform(30, 90) + delay_extra_segundos)

        dados.append({
            'id_chamada': 1000 + i,
            'timestamp_inicio': timestamp_inicio,
            'bairro': bairro,
            'categoria_real': categoria_real,
            'categoria_ia': categoria_ia,
            'categoria_humano': categoria_real,
            'timestamp_classificacao_ia': tempo_classificacao_ia,
            'timestamp_despacho': tempo_despacho,
        })

    df_simulado = pd.DataFrame(dados)
    nome_arquivo = 'logs_emergencia_simulados.csv'
    df_simulado.to_csv(nome_arquivo, index=False)

    print(f"✅ Arquivo '{nome_arquivo}' gerado com sucesso com {len(df_simulado)} registros.")
    print("Amostra dos dados gerados:")
    print(df_simulado.head())

    return nome_arquivo

# --------------------- FUNÇÃO PARA GERAR RELATÓRIO PDF ---------------------
def gerar_relatorio_pdf(indicadores, imagens, nome_pdf='relatorio_emergencias.pdf'):
    """
    Gera um relatório em PDF contendo os indicadores e os gráficos gerados.
    """
    c = canvas.Canvas(nome_pdf, pagesize=A4)
    width, height = A4
    margem = 40
    y = height - margem

    # Título
    c.setFont("Helvetica-Bold", 18)
    c.drawString(margem, y, "Relatório de Análise dos Logs de Emergência")
    y -= 40

    # Data/hora do relatório
    c.setFont("Helvetica", 10)
    c.drawString(margem, y, f"Gerado em: {datetime.now().strftime('%d/%m/%Y %H:%M')}")
    y -= 30

    # Indicadores
    c.setFont("Helvetica-Bold", 12)
    c.drawString(margem, y, "Indicadores de Performance:")
    y -= 20
    c.setFont("Helvetica", 11)
    for k, v in indicadores.items():
        c.drawString(margem, y, f"- {k}: {v}")
        y -= 17

    y -= 20
    c.setFont("Helvetica-Bold", 12)
    c.drawString(margem, y, "Gráficos Gerados:")
    y -= 20

    for img_path, legenda in imagens:
        # Adiciona legenda
        c.setFont("Helvetica", 11)
        c.drawString(margem, y, legenda)
        y -= 10

        # Adiciona a imagem
        try:
            img = ImageReader(img_path)
            iw, ih = img.getSize()
            fator = min((width-2*margem)/iw, 300/ih, 1.0)  # Ajusta tamanho máximo
            iw, ih = iw*fator, ih*fator
            if y-ih < 60:  # Se não couber, próxima página
                c.showPage()
                y = height - margem
            c.drawImage(img, margem, y-ih, width=iw, height=ih)
            y -= ih + 25
        except Exception as e:
            c.setFont("Helvetica", 9)
            c.drawString(margem, y, f"[Erro ao inserir imagem {img_path}: {e}]")
            y -= 20

    c.save()
    print(f"\n✅ Relatório PDF salvo como '{nome_pdf}'.")

# ---------------------- ANÁLISE DOS LOGS E GERAÇÃO DE GRÁFICOS ----------------------
def analisar_logs(nome_arquivo):
    """
    Carrega o arquivo de logs, calcula indicadores e gera gráficos para análise,
    salvando cada gráfico em uma pasta. Por fim, gera um relatório PDF.
    """
    print("\n--- Iniciando a Análise dos Dados ---")

    if not os.path.exists('graficos'):
        os.makedirs('graficos')
        print("Pasta 'graficos' criada para salvar as imagens.")

    df = pd.read_csv(nome_arquivo)

    for col in ['timestamp_inicio', 'timestamp_classificacao_ia', 'timestamp_despacho']:
        df[col] = pd.to_datetime(df[col])

    # --- Cálculo dos Indicadores ---
    df['tempo_resposta_ia'] = (df['timestamp_classificacao_ia'] - df['timestamp_inicio']).dt.total_seconds()
    df['tempo_total_despacho'] = (df['timestamp_despacho'] - df['timestamp_inicio']).dt.total_seconds()

    tempo_medio_resposta_ia = df['tempo_resposta_ia'].mean()
    acuracia_ia = (df['categoria_ia'] == df['categoria_humano']).mean() * 100
    tempo_medio_despacho = df['tempo_total_despacho'].mean()

    print("\n--- Indicadores de Performance ---")
    print(f"📈 Tempo Médio de Resposta da IA: {tempo_medio_resposta_ia:.2f} segundos")
    print(f"🎯 Acurácia na Classificação de Emergências: {acuracia_ia:.2f}%")
    print(f"🚑 Tempo Médio Total para Despacho: {tempo_medio_despacho:.2f} segundos")

    # --- Identificação de Gargalos com Visualizações ---
    print("\n--- Gerando e Salvando Gráficos ---")

    fig_hist = px.histogram(df, x='tempo_total_despacho', title='Distribuição do Tempo de Despacho (Geral)', labels={'tempo_total_despacho': 'Tempo (segundos)'})
    fig_hist.write_image("graficos/1_distribuicao_despacho.png")

    fig_resposta_ia = px.histogram(df, x='tempo_resposta_ia', nbins=40, title='Distribuição do Tempo de Resposta da IA', labels={'tempo_resposta_ia': 'Tempo de Resposta da IA (segundos)'})
    fig_resposta_ia.write_image("graficos/2_distribuicao_resposta_ia.png")

    fig_box_cat = px.box(df, x='categoria_humano', y='tempo_total_despacho', title='Análise de Gargalos por Tipo de Emergência', labels={'categoria_humano': 'Tipo de Emergência', 'tempo_total_despacho': 'Tempo (segundos)'})
    fig_box_cat.write_image("graficos/3_gargalos_por_categoria.png")

    fig_box_bairro = px.box(df, x='bairro', y='tempo_total_despacho', title='Análise de Gargalos por Bairro', labels={'bairro': 'Bairro', 'tempo_total_despacho': 'Tempo (segundos)'})
    fig_box_bairro.write_image("graficos/4_gargalos_por_bairro.png")

    print("\n✅ Gráficos salvos com sucesso na pasta 'graficos'.")

    # --- Gerar PDF ---
    indicadores = {
        'Tempo Médio de Resposta da IA': f"{tempo_medio_resposta_ia:.2f} segundos",
        'Acurácia na Classificação': f"{acuracia_ia:.2f} %",
        'Tempo Médio Total para Despacho': f"{tempo_medio_despacho:.2f} segundos"
    }
    imagens = [
        ("graficos/1_distribuicao_despacho.png", "Distribuição do Tempo de Despacho"),
        ("graficos/2_distribuicao_resposta_ia.png", "Distribuição do Tempo de Resposta da IA"),
        ("graficos/3_gargalos_por_categoria.png", "Gargalos por Categoria"),
        ("graficos/4_gargalos_por_bairro.png", "Gargalos por Bairro")
    ]
    gerar_relatorio_pdf(indicadores, imagens)

# ---------------------- EXECUÇÃO PRINCIPAL ----------------------
if __name__ == '__main__':
    # Passo 1: Gera o arquivo de dados
    arquivo_de_logs = gerar_dados_simulados()

    # Passo 2: Analisa o arquivo gerado e gera relatório PDF
    analisar_logs(arquivo_de_logs)

--- Iniciando a Geração de Dados Simulados ---
✅ Arquivo 'logs_emergencia_simulados.csv' gerado com sucesso com 1000 registros.
Amostra dos dados gerados:
   id_chamada    timestamp_inicio             bairro        categoria_real  \
0        1000 2025-05-16 03:59:00   Praia dos Amores  Acidente de Trânsito   
1        1001 2025-05-28 01:41:00   Praia dos Amores                 Queda   
2        1002 2025-05-20 15:41:00    Vila Industrial       Parada Cardíaca   
3        1003 2025-05-10 07:40:00  Parque das Nações              Incêndio   
4        1004 2025-05-29 22:34:00    Vila Industrial                 Queda   

           categoria_ia      categoria_humano timestamp_classificacao_ia  \
0  Acidente de Trânsito  Acidente de Trânsito 2025-05-16 03:59:10.242165   
1                 Queda                 Queda 2025-05-28 01:41:07.669275   
2       Parada Cardíaca       Parada Cardíaca 2025-05-20 15:41:09.308246   
3              Incêndio              Incêndio 2025-05-10 07:40:08.733984